## 1. Scoring Function

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score

def calculate_metrics(true_labels, predicted_labels,n_classes):
  """
  Calculates F1 score, precision, recall, and per-class AUC for a multiclass classification problem with 3 classes.

  Args:
    true_labels: A list of true class labels for each sample.
    predicted_labels: A list of predicted class labels for each sample.
    n_classes: total number of classes (>2)

  Returns:
    A dictionary containing the F1 score, precision, recall (macro-averaged), and per-class AUC.
  """

  # Check if the lengths of the lists are equal
  if len(true_labels) != len(predicted_labels):
    raise ValueError("Length of true labels and predicted labels must be equal.")

  # Calculate F1 score and macro-averaged precision and recall
  f1 = f1_score(true_labels, predicted_labels, average='macro')
  precision = precision_score(true_labels, predicted_labels, average='macro')
  recall = recall_score(true_labels, predicted_labels, average='macro')

  # Calculate per-class AUC
  auc_scores = []
  for class_label in range(n_classes):
    auc_scores.append(roc_auc_score(true_labels, predicted_labels, multi_class='ovo'))

  # Return metrics as a dictionary
  return {
      "f1_score": f1,
      "precision": precision,
      "recall": recall,
      "auc_per_class": auc_scores
  }

## 2. Load Training Data

In [ ]:
import pandas as pd

df = pd.read_csv("data/train_data.csv", index_col=0)
df.shape

In [ ]:
# Change datatypes

df['a'] = pd.to_datetime(df['a']).astype('int64')
df['b'] = pd.to_datetime(df['b']).astype('int64')
df['c'] = pd.to_datetime(df['c']).astype('int64')

## 3. Feature Engineering
Select the optimal features suggested by the algorithms.

In [ ]:
feature_cols = ['a',
  'b',
  'c',
  'd'
]  

# Add the columns to be predicted to the selection
select = feature_cols + ["to_be_predicted"]

# Drop the missing values
df.dropna()

# Drop the outliers
from scipy import stats

df = df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]

# Partition the dataset to features and predictions
X = df[feature_cols]

y = df.pop('to_be_predicted')

## 4. Train Model
Split X and y into training and testing sets and fit the model with data.


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=16)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

## 5. Evaluate Model

In [ ]:
X_res = X_test.assign(pnps_class=y_test).assign(pnps_p_class=y_pred)
X_res.head()

In [ ]:
from sklearn.preprocessing import label_binarize

# Example usage
# You need the labels to binarize
labels = ['a', 'b', 'c']
true_labels = label_binarize(y_test, classes=labels)
predicted_labels = label_binarize(y_pred, classes=labels)

metrics = calculate_metrics(true_labels, predicted_labels,n_classes=len(labels))

print(f"F1 Score: {metrics['f1_score']:.4f}")
print(f"Macro-averaged Precision: {metrics['precision']:.4f}")
print(f"Macro-averaged Recall: {metrics['recall']:.4f}")
print(f"AUC per Class: {metrics['auc_per_class']}")

## 6. Create the evaluation data

In [ ]:
edf = pd.read_csv("data/eval_data.csv", index_col=0)

X_eval = edf[feature_cols] # Features
y_eval = model.predict(X_eval)

In [ ]:
edf = edf.assign(to_be_predicted=y_eval)

In [ ]:
edf["to_be_predicted"].to_csv('results/my_results.csv')